In [1]:
import numpy as np 
def codeWords(m): 
    return [list(map(int, bin(i)[2:].zfill(m))) for i in range(2^m)]

def majority_decode(received_codeword, r, m):
    # Calculate the code parameters
    n = 2 ** m  # Total number of codewords
    k = k = sum([binomial(m, i) for i in range(r+1)])   # Number of message bits

    # Construct the generator matrix for the Reed-Muller code
    generator_matrix = construct_generator_matrix(r, m)

    # Calculate the syndrome vector
    syndrome = calculate_syndrome(received_codeword, generator_matrix)

    # Find the error locations using majority decoding
    error_locations = find_error_locations(syndrome, r, m)

    # Correct the errors in the received codeword
    corrected_codeword = correct_errors(received_codeword, error_locations)

    # Extract the message bits from the corrected codeword
    decoded_message = corrected_codeword[:k]

    return decoded_message


def eval2(codeword, j, r, m): 
    val = 1 
    for i in range(m): 
        tmp = codeword[i]
        if tmp == 0: 
            val *= (-1) ** bin(j)[2:].zfill(m)[i]
    return val 

def construct_generator_matrix(r, m):
    """
    Construct the generator matrix for the Reed-Muller code.

    Arguments:
    r -- The order of the Reed-Muller code.
    m -- The number of variables in the code.

    Returns:
    generator_matrix -- The generator matrix as a 2D numpy array.
    """
    # Calculate the code parameters
    n = 2 ** m  # Total number of codewords
    k = sum([binomial(m, i) for i in range(r + 1)])  # Number of message bits

    # Initialize the generator matrix as a zero matrix
    generator_matrix = np.zeros((k, n), dtype=int)

    # Generate the rows of the generator matrix
    for i in range(n):
        codeword = [int(bit) for bit in bin(i)[2:].zfill(m)]  # Convert the codeword index to binary
        for j in range(k):
            value = eval2(codeword, j, r, m)
            generator_matrix[j][i] = value

    return generator_matrix

def evaluate_reed_muller_function(codeword, j, r, m):
    """
    Evaluate the value of the j-th Reed-Muller function at the given codeword.

    Arguments:
    codeword -- The codeword as a list of binary digits.
    j -- The index of the Reed-Muller function to evaluate.
    r -- The order of the Reed-Muller code.
    m -- The number of variables in the code.

    Returns:
    value -- The value of the Reed-Muller function.
    """

    value = 1

    for i in range(m):
        variable = codeword[i]
        if variable == 0:
            value *= (-1) ** ((j >> (m - i - 1)) & 1)
    return value


def calculate_syndrome(received_codeword, generator_matrix):
    """
    Calculate the syndrome vector for the received codeword.

    Arguments:
    received_codeword -- The received codeword as a list of binary digits.
    generator_matrix -- The generator matrix as a 2D numpy array.

    Returns:
    syndrome -- The syndrome vector as a list of binary digits.
    """

    n = len(received_codeword)
    k = len(generator_matrix)

    syndrome = [0] * k

    for i in range(k):
        for j in range(n):
            syndrome[i] += received_codeword[j] * generator_matrix[i][j]
        syndrome[i] %= 2
    
    return syndrome


def find_error_locations(syndrome, r, m):
    """
    Find the error locations using majority decoding.

    Arguments:
    syndrome -- The syndrome vector as a list of binary digits.
    r -- The order of the Reed-Muller code.
    m -- The number of variables in the code.

    Returns:
    error_locations -- The error locations as a list of binary digits.
    """

    error_locations = []

    for i, bit in enumerate(syndrome):
        if bit == 1:
            codeword_index = i // (2 ** (m - r))
            error_locations.append(bin(codeword_index)[2:].zfill(m))

    return error_locations


def correct_errors(received_codeword, error_locations):
    """
    Correct the errors in the received codeword.

    Arguments:
    received_codeword -- The received codeword as a list of binary digits.
    error_locations -- The error locations as a list of binary digits.

    Returns:
    corrected_codeword -- The corrected codeword as a list of binary digits.
    """

    corrected_codeword = received_codeword.copy()

    for location in error_locations:
        index = int(location, 2)
        corrected_codeword[index] = 1 - corrected_codeword[index]

    return corrected_codeword


In [3]:
q = 2 
m = 3 
r = 1 
n = q^m 

F = GF(2) 

In [5]:
def codeWords(m): 
    return [list(map(int, bin(i)[2:].zfill(m))) for i in range(2^m)]

def generateG(): 
    X = matrix(F, codeWords(m)).T
    G = [[1 if j == 0 else 0 for i in range(n)] for j in range(m + 1)]
    i = m -1
    j = 1
    for _ in range(m): 
        G[j] = X[i]
        j += 1
        i -= 1
    return matrix(F, G) 

G = generateG()
H = G.echelon_form()

In [8]:
C = codes.BinaryReedMullerCode(2, 4) 

In [9]:
C.generator_matrix()

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]
[0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]
[0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1]
[0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1]
[0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1]
[0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1]
[0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]

In [10]:
def generator_matrix():
    C = self.code()
    base_field = C.base_field()
    order = C.order()
    num_of_var = C.number_of_variables()
    q = base_field.cardinality()
    points = base_field**num_of_var
    
    matrix_list = []
    max_individual_degree = min(order, (q - 1))
    for degree in range(order + 1):
        exponents = Subsets(list(range(num_of_var)) * max_individual_degree,
                            degree, submultiset=True)
        matrix_list += [[reduce(mul, [x[i] for i in exponent], 1)
                         for x in points] for exponent in exponents]
    M = matrix(base_field, matrix_list)
    M.set_immutable()
    return M
    

In [16]:
from itertools import chain, combinations
from functools import reduce
from operator import mul
from sage.matrix.constructor import matrix

def generator_matrix(order, num_of_var, base_field):
    # Obtain the cardinality of the base field
    q = base_field.cardinality()
    
    # Compute the number of points based on the number of variables
    points = base_field ** num_of_var
    
    # Initialize an empty list to store the matrix elements
    matrix_list = []
    
    # Determine the maximum individual degree based on the order and field cardinality
    max_individual_degree = min(order, (q - 1))
    
    # Iterate over each degree from 0 to the given order
    for degree in range(order + 1):
        # Generate all possible combinations of exponents for the current degree
        # The exponents determine the positions of elements from points to include in the matrix
        # The combinations are generated by selecting 'degree' number of variables from 'num_of_var' with repetitions
        exponents = chain.from_iterable(combinations(range(num_of_var), degree) for _ in range(max_individual_degree))
        
        # Compute the matrix elements for the current degree
        # For each combination of exponents, calculate the product of corresponding elements from points
        # and append the product to matrix_list
        matrix_list += [[reduce(mul, [x[i] for i in exponent], 1) for x in points] for exponent in exponents]
    
    # Create the matrix M using the base field and the matrix list
    M = matrix(base_field, matrix_list)
    
    # Set the matrix as immutable to prevent accidental modifications
    M.set_immutable()
    
    # Return the generator matrix M
    return M


In [19]:
order = 2 
num_of_var = 4 
base_field = FiniteField(2) 

M = generator_matrix(order, num_of_var, base_field) 